In [1]:
import pickle
import pandas as pd
import numpy as np
from imp import reload
import h5py
import shutil
import torch
import torch.nn as nn
from torch.nn import init
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader
from itertools import chain
from sklearn.model_selection import KFold
from progiter import ProgIter
from sklearn.model_selection import train_test_split
from mymodule import VAE_model

random_state=34

In [2]:
reload(VAE_model)

<module 'mymodule.VAE_model' from '/tf/notebooks/code_for_pub/mymodule/VAE_model.py'>

In [3]:
# all drugs
with open('/tf/notebooks/code_for_pub/smiles_files/smiles_drugcombANDchembl26.pickle','rb') as f:
    a = pickle.load(f)

In [5]:
with open('/tf/notebooks/code_for_pub/smiles_files/smiles_drugcomb_BY_cid_duplicated.pickle','rb') as f:
    b = pickle.load(f)

In [6]:
processor = VAE_model.ProcessSMILES(smiles_chembl=a, smiles_dc=b)


1795483 unique SMILES read in 11.0767 seconds


### 256 bit

In [7]:
# dataprep
df = processor.one_hot_encoded(processor.smiles)

one-hot-encoded 1795483 SMILES in 225.969 seconds


In [8]:
batch_size = 1800
patience = 3
n_epochs = 7
charset = processor.charset
dtype=torch.cuda.FloatTensor

#best = torch.load('/tf/notebooks/code_for_pub/_logs_as_python_files/vae_training_logs/model_best.pth.tar')
encoder = VAE_model.MolEncoder(i=140, o=256, c=len(charset))
#encoder.load_state_dict(best['encoder'])
encoder.apply(VAE_model.initialize_weights)
decoder = VAE_model.MolDecoder(i=140, o=256, c=len(charset))
decoder.apply(VAE_model.initialize_weights)
#decoder.load_state_dict(best['decoder'])
encoder.cuda()
decoder.cuda()
best_loss = 1e6
#best_loss = best['avg_val_loss']

optimizer = torch.optim.Adam(chain(encoder.parameters(), decoder.parameters()), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                       factor=0.2,
                                                       patience=patience,
                                                       mode='min', 
                                                       min_lr=1e-6)

kf = KFold(n_splits=5, shuffle=True, random_state=random_state)        
for ind,(train_index, test_index) in enumerate(kf.split(df)):
    
    # make loader train
    dt = torch.from_numpy(df[train_index])
    train = TensorDataset(dt, torch.zeros(dt.size()[0]))
    train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)

    # make loader test
    dte = torch.from_numpy(df[test_index])
    test = TensorDataset(dte, torch.zeros(dte.size()[0]))
    val_loader = DataLoader(test, batch_size=batch_size, shuffle=True)
    del dt, dte, train, test
    
    # for other folds we re-use previous best model's weights
    if ind != 0:
        encoder = VAE_model.MolEncoder(i=140, o=256, c=len(charset))
        best = torch.load('/tf/notebooks/code_for_pub/_logs_as_python_files/vae_training_logs/model_best.pth.tar')
        encoder.load_state_dict(best['encoder'])
        decoder = VAE_model.MolDecoder(i=140, o=256, c=len(charset))
        decoder.load_state_dict(best['decoder'])
        encoder.cuda()
        decoder.cuda()

        optimizer = torch.optim.Adam(chain(encoder.parameters(), decoder.parameters()), lr=1e-3)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                               factor=0.1,
                                                               patience=patience,
                                                               mode='min', 
                                                               min_lr=1e-6a)
    
    
    
    # train for n epochs
    for epoch in range(n_epochs):
        print(f'outer fold:{ind}, epoch:{epoch}')
        VAE_model.train_model(train_loader, encoder, decoder, optimizer, dtype)
        avg_val_loss = VAE_model.validate_model(val_loader, encoder, decoder, dtype)

        scheduler.step(avg_val_loss)

        is_best = avg_val_loss < best_loss

        if is_best:
            best_loss = avg_val_loss
        VAE_model.save_checkpoint({
            'epoch': epoch,
            'encoder': encoder.state_dict(),
            'decoder': decoder.state_dict(),
            'charset': charset,
            'avg_val_loss': avg_val_loss,
            'optimizer': optimizer.state_dict(),
        }, is_best, size=256)

    del train_loader, encoder, decoder, scheduler, optimizer 


NameError: name 'charset' is not defined

### 16 bit

In [5]:
processor = VAE_model.Process_SMILES()

1795483 unique SMILES read in 2.6206 seconds


In [6]:
df= processor.one_hot_encode_all(processor.smiles)

one-hot-encoded 1795483 SMILES in 253.2625 seconds


In [7]:
batch_size = 1900
patience = 2
n_epochs = 10
dtype=torch.cuda.FloatTensor
charset = processor.charset
latent_space_size = 16

best = torch.load(f'/tf/notebooks/code_for_pub/_logs_as_python_files/vae_training_logs/{str(latent_space_size)}_model_best.pth.tar')
#encoder.apply(VAE_model.initialize_weights)
#decoder = VAE_model.MolDecoder(i=latent_space_size, o=140, c=len(charset))
#decoder.apply(VAE_model.initialize_weights)
encoder = VAE_model.MolEncoder(i=140, o=latent_space_size, c=len(charset))
encoder.load_state_dict(best['encoder'])

decoder = VAE_model.MolDecoder(i=latent_space_size, o=140, c=len(charset))
decoder.load_state_dict(best['decoder'])

encoder.cuda()
decoder.cuda()
best_loss = 1e6

optimizer = torch.optim.Adam(chain(encoder.parameters(), decoder.parameters()), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                       factor=0.8,
                                                       patience=patience,
                                                       mode='min', 
                                                       min_lr=1e-6)

kf = KFold(n_splits=5, shuffle=True, random_state=random_state)        
for ind,(train_index, test_index) in enumerate(kf.split(df)):
    
    # make loader train
    dt = torch.from_numpy(df[train_index])
    train = TensorDataset(dt, torch.zeros(dt.size()[0]))
    train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)

    # make loader test
    dte = torch.from_numpy(df[test_index])
    test = TensorDataset(dte, torch.zeros(dte.size()[0]))
    val_loader = DataLoader(test, batch_size=batch_size, shuffle=True)
    del dt, dte, train, test
    
    # for other folds we re-use previous best model's weights
    if ind != 0:
        encoder = VAE_model.MolEncoder(i=140, o=latent_space_size, c=len(charset))
        best = torch.load(f'/tf/notebooks/code_for_pub/_logs_as_python_files/vae_training_logs/{str(latent_space_size)}_model_best.pth.tar')
        encoder.load_state_dict(best['encoder'])
        decoder = VAE_model.MolDecoder(i=latent_space_size, o=140, c=len(charset))
        decoder.load_state_dict(best['decoder'])
        encoder.cuda()
        decoder.cuda()

        optimizer = torch.optim.Adam(chain(encoder.parameters(), decoder.parameters()), lr=1e-3)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                               factor=0.2,
                                                               patience=patience,
                                                               mode='min', 
                                                               min_lr=1e-6)
    
    
    
    # train for n epochs
    for epoch in ProgIter(range(n_epochs), verbose=1, total=n_epochs):
        print(f'outer fold:{ind}, epoch:{epoch}')
        VAE_model.train_model(train_loader, encoder, decoder, optimizer, dtype)
        avg_val_loss = VAE_model.validate_model(val_loader, encoder, decoder, dtype)

        scheduler.step(avg_val_loss)

        is_best = avg_val_loss < best_loss

        if is_best:
            best_loss = avg_val_loss
        VAE_model.save_checkpoint({
            'epoch': epoch,
            'encoder': encoder.state_dict(),
            'decoder': decoder.state_dict(),
            'charset': charset,
            'avg_val_loss': avg_val_loss,
            'optimizer': optimizer.state_dict(),
        }, is_best, size = latent_space_size)

    del train_loader, encoder, decoder, scheduler, optimizer 


  0/10... rate=0 Hz, eta=?, total=0:00:00, wall=16:29 UTCouter fold:0, epoch:0
t = 100, loss = 1.54854369
t = 200, loss = 1.55469525
t = 300, loss = 1.56053662
t = 400, loss = 1.50880802
t = 500, loss = 1.53006780
t = 600, loss = 1.54756558
t = 700, loss = 1.54510748
average validation loss: 1.52949405
  1/10... rate=0.00 Hz, eta=1:53:39, total=0:12:37, wall=16:42 UTCouter fold:0, epoch:1
t = 100, loss = 1.54035807
t = 200, loss = 1.49055052
t = 300, loss = 1.53780365
t = 400, loss = 1.53209603
t = 500, loss = 1.79145467
t = 600, loss = 1.58308709
t = 700, loss = 1.49396610
average validation loss: 1.51151991
  2/10... rate=0.00 Hz, eta=1:40:57, total=0:25:14, wall=16:55 UTCouter fold:0, epoch:2
t = 100, loss = 1.49950171
t = 200, loss = 1.47682941
t = 300, loss = 1.50885391
t = 400, loss = 1.51407170
t = 500, loss = 1.51095748
t = 600, loss = 1.49812484
t = 700, loss = 1.58757770
average validation loss: 1.48061466
  3/10... rate=0.00 Hz, eta=1:28:17, total=0:37:50, wall=17:07 UTCoute

average validation loss: 1.11699271
  6/10... rate=0.00 Hz, eta=0:50:23, total=1:15:35, wall=21:59 UTCouter fold:2, epoch:6
t = 100, loss = 1.11760032
t = 200, loss = 1.10683370
t = 300, loss = 1.10707080
t = 400, loss = 1.12318027
t = 500, loss = 1.08443213
t = 600, loss = 1.03949726
t = 700, loss = 1.15147984
average validation loss: 1.11728251
  7/10... rate=0.00 Hz, eta=0:37:47, total=1:28:11, wall=22:11 UTCouter fold:2, epoch:7
t = 100, loss = 1.06308961
t = 200, loss = 1.08312571
t = 300, loss = 1.07712138
t = 400, loss = 1.04331720
t = 500, loss = 1.05978858
t = 600, loss = 1.13209796
t = 700, loss = 1.09352243
average validation loss: 1.11491477
  8/10... rate=0.00 Hz, eta=0:25:11, total=1:40:47, wall=22:24 UTCouter fold:2, epoch:8
t = 100, loss = 1.14065921
t = 200, loss = 1.15282583
t = 300, loss = 1.09161639
t = 400, loss = 1.12661481
t = 500, loss = 1.14359820
t = 600, loss = 1.16311002
t = 700, loss = 1.07669079
average validation loss: 1.10470498
  9/10... rate=0.00 Hz, e